In [1]:
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd

input1 = "/Users/surthi/Documents/cs230project/data/allSingles.json"
def read(filePath):
    contents = open(filePath, "r").read() 
    json_data = [json.loads(str(item)) for item in contents.strip().split('\n')]
    df = json_normalize(json_data)
    return df
df1 = read(input1)

In [2]:
# for singles, we dont need isProminent flags
df1.columns
df2 = df1.drop(['isProminent','isProminentByItself','isProminentByParent','isSingleCompany'], axis=1)
# df2 = df2[['articleLength', 'countOfDistinctIndustry', 'countOfOccurence', 'countOfOccurenceOfAliases', 
#       'countOfOccurenceOfNamePreserveList', 'countOfSignificantStrings', 'countOfUrlMatch', 
#       'countryMatchByOverlapScore', 'directCityMatchScore', 'directCountryMatchScore', 'directStateMatchScore',
#      'exactMatch', 'fieldMatch', 'isIndustry', 'isTickerMatch', 'isUrlMatched', 'personScore', 'stateMatchByOverlapScore', 'sumOfIndustry', 'w2vScore','isGold' ]].replace(-1000.0, np.nan)
# convert boolean features to 0's and 1's:
df2[['isIndustry', 'fieldMatch', 'isTickerMatch', 'isUrlMatched', 'isGold','exactMatch']] *= 1.0
df2.columns

# stratify split
from sklearn.model_selection import train_test_split
train_rf, test_rf = train_test_split(df2, test_size=0.2, stratify=df2["isGold"])

# normalize
X_train_rf = train_rf.loc[:, train_rf.columns != "isGold"].drop(["companyId", "mentionMatched", "articleId"], axis=1)
y_train_rf = train_rf["isGold"]
X_test_rf = test_rf.loc[:, test_rf.columns != "isGold"].drop(["companyId", "mentionMatched", "articleId"], axis=1)
y_test_rf = test_rf["isGold"]

In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from time import time
from scipy.stats import randint as sp_randint

# build a classifier
clf = RandomForestClassifier(n_estimators=20)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, 5],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 50
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search, cv=5)

start = time()
random_search.fit(X_train_rf, y_train_rf)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# df2.isnull().sum()
rf = random_search.best_estimator_
print(random_search.best_params_)
d = dict(zip(X_train_rf.columns, rf.feature_importances_))
for w in sorted(d, key=d.get, reverse=True):
    print(w, d[w])

rf_model = random_search.best_estimator_
y_pred_rf = rf_model.predict(X_test_rf)

from sklearn.metrics import classification_report
print(classification_report(y_test_rf, np.asarray(y_pred_rf >= 0.5)))
print(len(y_pred_rf[y_pred_rf == 0.0]), len(y_pred_rf[y_pred_rf == 1.0]))

RandomizedSearchCV took 9.31 seconds for 50 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.868 (std: 0.017)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'max_features': 10, 'min_samples_split': 8}

Model with rank: 2
Mean validation score: 0.866 (std: 0.013)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'max_features': 6, 'min_samples_split': 2}

Model with rank: 3
Mean validation score: 0.863 (std: 0.010)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'max_features': 8, 'min_samples_split': 7}

Model with rank: 3
Mean validation score: 0.863 (std: 0.016)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 10, 'min_samples_split': 4}

Model with rank: 3
Mean validation score: 0.863 (std: 0.019)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'max_features': 6, 'min_samples_split': 6}

{'bootstrap': False, 'criterion': 'gini', 'max

In [5]:
df3 = df2[['articleLength', 'countOfDistinctIndustry', 'countOfOccurence', 'countOfOccurenceOfAliases', 
      'countOfOccurenceOfNamePreserveList', 'countOfSignificantStrings', 'countOfUrlMatch', 
      'countryMatchByOverlapScore', 'directCityMatchScore', 'directCountryMatchScore', 'directStateMatchScore',
     'exactMatch', 'fieldMatch', 'isIndustry', 'isTickerMatch', 'isUrlMatched', 'personScore', 'stateMatchByOverlapScore', 'sumOfIndustry', 'w2vScore','isGold' ]].replace(-1000.0, np.nan)

df3['countryMatchByOverlapScore'].fillna(df3['countryMatchByOverlapScore'].mean(), inplace = True)
df3['directCityMatchScore'].fillna(df3['directCityMatchScore'].mean(), inplace = True)
df3['directCountryMatchScore'].fillna(df3['directCountryMatchScore'].mean(), inplace = True)
df3['directStateMatchScore'].fillna(df3['directStateMatchScore'].mean(), inplace = True)
df3['personScore'].fillna(df3['personScore'].mean(), inplace = True)
df3['stateMatchByOverlapScore'].fillna(df3['stateMatchByOverlapScore'].mean(), inplace = True)
df3['w2vScore'].fillna(df3['w2vScore'].mean(), inplace = True)
df3.isnull().sum()

# stratify split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df3, test_size=0.2, stratify=df3["isGold"])

# normalize
X_train_svc = train.loc[:, train.columns != "isGold"]
y_train_svc = train["isGold"]
X_test_svc = test.loc[:, test.columns != "isGold"]
y_test_svc = test["isGold"]
X_normalized_train_svc=(X_train_svc-X_train_svc.mean())/X_train_svc.std()
X_normalized_test_svc=(X_test_svc-X_train_svc.mean())/X_train_svc.std()

# checking stratified counts
z = y_train_svc.to_frame()
print(z[z["isGold"] == 1.0].count(), z[z["isGold"] == 0.0].count())
z1 = y_test_svc.to_frame()
print(z1[z1["isGold"] == 1.0].count())
print(z1[z1["isGold"] == 0.0].count())

from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import tree, metrics, model_selection, preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

svr = svm.SVC(max_iter=100,random_state=0)
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict( C=C_range, gamma=gamma_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
grid = GridSearchCV(svr, param_grid=param_grid, cv=cv)
grid.fit(X_normalized_train_svc, y_train_svc)
y_pred = grid.predict(X_normalized_train_svc)

isGold    986
dtype: int64 isGold    200
dtype: int64
isGold    247
dtype: int64
isGold    50
dtype: int64


/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

...

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/

ValueError: Found input variables with inconsistent numbers of samples: [297, 1186]

In [8]:
svc_model = grid.best_estimator_
y_pred_svc = svc_model.predict(X_normalized_test_svc)

from sklearn.metrics import classification_report
print(classification_report(y_test_svc, np.asarray(y_pred_svc >= 0.9)))
print(len(y_pred_svc[y_pred_svc == 0.0]), len(y_pred_svc[y_pred_svc == 1.0]))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        50
         1.0       0.83      1.00      0.91       247

   micro avg       0.83      0.83      0.83       297
   macro avg       0.42      0.50      0.45       297
weighted avg       0.69      0.83      0.76       297

0 297


/Users/surthi/Documents/mygitrepo/cs230-code-examples/tensorflow/nlp/cs230/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
